# 数据集概览
## Load datasets 

In [1]:
from others import load_all_dataset, rename_dataset
X_train, y_train, X_test, y_test = load_all_dataset(show=False)
import numpy as np
np.set_printoptions(edgeitems=5,
                    linewidth=1000,
                    formatter={"float":lambda x: "{:.3f}".format(x)})

Train data
Test data


## 训练集
### City A (source)

In [ ]:
print("source labeled (City A weak & failure):", X_train.source.shape)
# print(np.where(np.isnan(X_train.source[0])))
print("|- labels:", y_train.source.shape)
print(" |- weak=0:", y_train.source[y_train.source==0].shape)
print(" |- failure=1:", y_train.source[y_train.source==1].shape)
print("source background (City A good):", X_train.source_bkg.shape)

### City B (target)

In [8]:
print("target labeled (City B weak & failure):", X_train.target.shape)
print("|- labels:", y_train.target.shape)
print(" |- weak=0:", y_train.target[y_train.target==0].shape)
print(" |- failure=1:", y_train.target[y_train.target==1].shape)
print("target background (City B good):", X_train.target_bkg.shape)

target labeled (City B weak & failure): (438, 672, 10)
|- labels: (438,)
 |- weak=0: (349,)
 |- failure=1: (89,)
target background (City B good): (29592, 672, 10)


## 测试集
### City B (target)

In [9]:
print("target labeled (City B weak & failure):", X_test.target.shape)
print("|- labels:", y_test.target.shape)
print(" |- weak=0:", y_test.target[y_test.target==0].shape)
print(" |- failure=1:", y_test.target[y_test.target==1].shape)
print("target background (City B good):", X_test.target_bkg.shape)

target labeled (City B weak & failure): (17758, 672, 9, 1)
|- labels: (17758,)
 |- weak=0: (15464,)
 |- failure=1: (2294,)
target background (City B good): (47275, 672, 9, 1)


# Residual Neural Network
## 预处理

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from feature_extractor import FeatureExtractor

fe = FeatureExtractor()

[X_source, X_source_bkg, X_target, X_target_unlabeled, X_target_bkg,
    y_source, y_target, X_test] = rename_dataset(
    fe, X_train, y_train, X_test, y_test, show_imbalance=0)


==== TRAIN SET ====
  | X_source: (46110, 672, 9, 1) ; y_source: (46110,)


/home/hbai/Codes/HuaweiRAMP/submissions/haobai_dnn/feature_extractor.py:52: RuntimeWarning: Mean of empty slice
  nanmean.append(np.nanmean(x, axis=0))


A | X_source_bkg: (50862, 672, 9, 1)
----
  | X_target: (438, 672, 9, 1) ; y_target: (438,)
B | X_target_bkg: (29592, 672, 9, 1)
  | X_target_unlabeled: (8202, 672, 9, 1)
==== TEST SET ====
  | X_test.target: (17758, 672, 9, 1) ; y_test.target: (17758,)
B | X_test.target_bkg: (47275, 672, 9, 1)
  | X_test.target_unlabeled: None


In [ ]:
# 去除NaN
from numpy import newaxis
class FeatureExtractor:

    def __init__(self):
        pass

    def transform(self, X):
        ''' Replace NaN by 0 and flatten the matrix to size (sample, 6720).
        Executed on every input data (i.e., source, bkg, target) and passed
        the resulting arrays to `fit`and `predict` methods in :class: Classifier

        Parameters
        ----------
        `X`: ndarray of (sample, 672, 10)
            3D input dataset(sample, time, features)
        
        Returns
        -------
        `X`: ndarray of (sample, 6720)
            The filtered dataset
        '''
        np.nan_to_num(X, copy=False)
        return X[:,:,:,newaxis]

fe = FeatureExtractor()

# 重命名
from copy import deepcopy
# 训练集
print("==== TRAIN SET ====")
X_source = deepcopy( fe.transform(X_train.source) )
print("  | X_source:", X_source.shape, end=" ; ")
y_source = deepcopy( y_train.source )
print("y_source:", y_source.shape)
X_source_bkg = deepcopy( fe.transform(X_train.source_bkg) )
print("A | X_source_bkg:", X_source_bkg.shape)
X_target = deepcopy( fe.transform(X_train.target) )
print("----")
print("  | X_target:", X_target.shape, end=" ; ")
y_target = deepcopy( y_train.target )
print("y_target:", y_target.shape)
X_target_bkg= deepcopy( fe.transform(X_train.target_bkg) )
print("B | X_target_bkg:", X_target_bkg.shape)
X_target_unlabeled = deepcopy( fe.transform(X_train.target_unlabeled) )
print("  | X_target_unlabeled:", X_target_unlabeled.shape)
# 测试集
print("==== TEST SET ====")
X_test.target = fe.transform(X_test.target)
print("  | X_test.target:", X_test.target.shape, end=" ; ")
print("y_test.target:", y_test.target.shape)
X_test.target_bkg = fe.transform(X_test.target_bkg)
print("B | X_test.target_bkg:", X_test.target_bkg.shape)
print("  | X_test.target_unlabeled:", X_test.target_unlabeled)

In [3]:
# 将数据集转换为TensorFlow格式
train_dataset = tf.data.Dataset.from_tensor_slices((X_source, y_source)).batch(16)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_target, y_target)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.target, y_test.target))

# 额外操作
train_dataset = train_dataset.map( lambda x, y: (tf.image.random_flip_left_right(x), y) )
train_dataset = train_dataset.repeat()
valid_dataset = valid_dataset.repeat()

## 搭建网络模型
参考资料

1. [Introduction to ResNet in TensorFlow 2](https://adventuresinmachinelearning.com/introduction-resnet-tensorflow-2/)

In [4]:
def res_net_block(filters, conv_size, input_data):
    ''' A residual block of 3 layers
    '''
    # 1st layer with batch normalization
    x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
    x = layers.BatchNormalization()(x)
    # 2nd layer with batch normalization, but no activation function
    x = layers.Conv2D(filters, conv_size, activation=None, padding='same')(x)
    x = layers.BatchNormalization()(x)
    # 3rd layer is residual addition with an activation function
    x = layers.Add()([x, input_data])
    x = layers.Activation('relu')(x)
    return x

In [5]:
# Layer to be used as an entry point into a Network (a graph of layers).
# https://keras.io/api/layers/core_layers/input/
inputs = tf.keras.Input(shape=(672, 9, 1), name="Input_Layer")
# inputs = layers.InputLayer(input_shape=(672, 10), name="Input_Layer")

# 2D convolution layer (e.g. spatial convolution over images).
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
# filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
# kernel_size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window.
x = layers.Conv2D(32, 2, activation='relu', name="Layer01")(inputs)
x = layers.Conv2D(64, 2, activation='relu', name="Layer02")(x)

# Max pooling operation for 2D spatial data.
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
# pool_size: integer or tuple of 2 integers, window size over which to take the maximum.
x = layers.MaxPooling2D(2, name="Layer03")(x)

num_res_net_blocks = 10 # 10个ResNet blocks
for i in range(num_res_net_blocks):
    x = res_net_block(64, 2, x)

# [Final layers] a standard CNN layer
x = layers.Conv2D(64, 2, activation='relu', name="Layer-4")(x)
# [Final layers] GAP layer
x = layers.GlobalAveragePooling2D(name="Layer-3")(x)
# [Final layers] dense classification layers
# Just your regular densely-connected NN layer.
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
# units: Positive integer, dimensionality of the output space.
x = layers.Dense(256, activation='relu', name="Layer-2")(x)
# [Final layers] dropout layer
x = layers.Dropout(0.5, name="Layer-1")(x)
# [Final layers] dense classification layers
outputs = layers.Dense(1, activation='softmax', name="Output_Layer")(x)
res_net_model = tf.keras.Model(inputs, outputs)

res_net_model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Layer (InputLayer)        [(None, 672, 9, 1)]  0                                            
__________________________________________________________________________________________________
Layer01 (Conv2D)                (None, 671, 8, 32)   160         Input_Layer[0][0]                
__________________________________________________________________________________________________
Layer02 (Conv2D)                (None, 670, 7, 64)   8256        Layer01[0][0]                    
__________________________________________________________________________________________________
Layer03 (MaxPooling2D)          (None, 335, 3, 64)   0           Layer02[0][0]                    
______________________________________________________________________________________________

## 训练模型
'binary_crossentropy' 效果比 'categorical_crossentropy' 稍好。后者计算的loss直接=0.

In [6]:
import datetime as dt
callbacks = [
    # Write TensorBoard logs to `./logs` directory
    tf.keras.callbacks.TensorBoard(log_dir='./log/{}'.format(
        dt.datetime.now().strftime("%Y-%m-%d-%H-%M ResNet")), write_images=True),
    ]
res_net_model.compile(optimizer=tf.keras.optimizers.Adam(),
                      loss='binary_crossentropy',
                      metrics=[tf.keras.metrics.Precision(),
                               # tf.keras.metrics.PrecisionAtRecall(recall=0.1),
                               "acc",
                      ]
                     )
res_net_model.fit(train_dataset, epochs=100, steps_per_epoch=200,
          validation_data=valid_dataset,
          validation_steps=3, callbacks=callbacks)

/home/hbai/anaconda3/envs/huawei/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/100
200/200 [==============================] - 37s 166ms/step - loss: 4638.3555 - precision: 0.1081 - acc: 0.1081 - val_loss: 1539.5272 - val_precision: 0.2917 - val_acc: 0.2917
Epoch 2/100
200/200 [==============================] - 32s 161ms/step - loss: 11105.8760 - precision: 0.0916 - acc: 0.0916 - val_loss: 19857.4512 - val_precision: 0.2917 - val_acc: 0.2917
Epoch 3/100
200/200 [==============================] - 32s 162ms/step - loss: 29912.4648 - precision: 0.0797 - acc: 0.0797 - val_loss: 1294.5199 - val_precision: 0.2917 - val_acc: 0.2917
Epoch 4/100
200/200 [==============================] - 32s 159ms/step - loss: 9215.5479 - precision: 0.0828 - acc: 0.0828 - val_loss: 4769.1187 - val_precision: 0.2917 - val_acc: 0.2917
Epoch 5/100
200/200 [==============================] - 33s 165ms/step - loss: 2778.1621 - precision: 0.0791 - acc: 0.0791 - val_loss: 1542.6415 - val_precision: 0.2917 - val_acc: 0.2917
Epoch 6/100
200/200 [==============================] - 32s 159ms/st

In [7]:
print("X_test.target.shape:", X_test.target.shape)
# X = X_test.target.reshape(X_test.target.shape[0], -1)
# print(X.shape)
y_pred = res_net_model.predict(X_test.target).transpose()
print("Predicted:", y_pred, y_pred.shape)
print("True:      ", y_test.target, y_test.target.shape)

X_test.target.shape: (17758, 672, 9, 1)
Predicted: [[1.000 1.000 1.000 1.000 1.000 ... 1.000 1.000 1.000 1.000 1.000]] (1, 17758)
True:       [0.000 0.000 0.000 0.000 0.000 ... 0.000 1.000 0.000 0.000 1.000] (17758,)


## 查看Tensorboard

In [ ]:
%tensorboard